In [1]:
import requests
import json
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pandas as pd

In [2]:
# wait
def wait(driver, selector_name):
    while (True):
        try:
            myElem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, selector_name)))
            if myElem is not None:
                break
        except TimeoutException:
            break

In [3]:
# scroll
def scroll(driver):
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
main_url = "https://soundcloud.com/discover"
set_link = []

In [5]:
# Drive the chrome to url link and return soup with html parser
def drive_to_url_page(driver, url):
    driver.get(url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [13]:
#Driver to soundcloud website
driver = webdriver.Chrome()
soup = drive_to_url_page(driver, main_url)
links = soup.find_all("a", {"class": "sc-link-secondary sc-link-light playableTile__usernameHeading audibleTile__usernameHeading sc-truncate sc-text-h4"})
for item in links:
    user_url = "https://soundcloud.com" + item.get("href")
    if user_url not in set_link:
        set_link.append(user_url)

#Get the all person in set_link with his/her following information
for i in range(0, len(set_link) - 1):
    #Set number of user
    if len(set_link) >= 20:
        break;
        
    following_url = set_link[i] + "/following"
    soup = drive_to_url_page(driver, following_url)
    link_following_user = soup.find_all("a", {"class":"userBadgeListItem__heading sc-type-small sc-text-h4 sc-link-dark sc-link-primary sc-truncate"})
    for item in link_following_user:
        user_url = "https://soundcloud.com" + item.get("href")
        if user_url not in set_link:
            set_link.append(user_url)
    
    # print(len(link_following_user))
    
driver.close()

In [14]:
print(len(set_link))

1147


In [15]:
#Count all playlist in user_url/sets (count by <li> tag)
def count_playlist_in_one_person(driver, url_of_user):
    playlist_url = str(url_of_user) + '/sets'
    soup = drive_to_url_page(driver, playlist_url)
    playlist_count = soup.find_all("li", {"class":"soundList__item"})
    return len(playlist_count)

In [16]:
def get_all_user_data(driver, set_link):
    #Get user id in SoundCloud/Discover by get meta tag in HEAD html
    set_data_user = []
    driver = webdriver.Chrome()

    for link_user in set_link:
        one_data_user = []
        driver.get(link_user)
        wait(driver, 'infoStats__value sc-font-light')
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        #Get USER INFO
        id_user = soup.find("meta", property="twitter:app:url:googleplay")
        name_user = soup.find("meta", property="og:title")
        url_user = soup.find("meta", property="og:url")
        info_start = soup.find_all("div", {"class":"infoStats__value sc-font-light"})
        
        likeTrack_count = soup.find_all("span", {"class":"sidebarHeader__actualTitle sc-text-h3"})
        playlist_count = count_playlist_in_one_person(driver, link_user)
        # more_info = soup.find("h3", {"class": "profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary sc-mt-1x"}).getText().Strip()

        #Add to one_data_user set
        one_data_user.append(id_user["content"].strip("soundcloud://users:"))
        one_data_user.append(name_user["content"])
        one_data_user.append(url_user["content"])
        one_data_user.append(playlist_count)
        # one_data_user.append(more_info)
        one_data_user.append(info_start[0].getText())   #followers
        one_data_user.append(info_start[1].getText())   #following
        one_data_user.append(info_start[2].getText())   #Track count
        one_data_user.append(likeTrack_count[1].getText().strip(' likes'))
        #Add one data user to set data 
        set_data_user.append(one_data_user)
    return set_data_user

In [17]:
set_data_user = get_all_user_data(driver, set_link)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=95.0.4638.69)


In [ ]:
#write set data user to file <'user.csv'>
df = pd.DataFrame(set_data_user,columns=['User ID', 
                                         'User Name', 
                                         'User Link', 
                                         'Playlist Count', 
                                         'Followers', 
                                         'Following', 
                                         'Track Count', 
                                         'Like Tracks'])
df.to_csv('user.csv')

In [ ]:
def getTracks(set_user):
    driver = webdriver.Chrome()
    set_tracks = set()
    set_link_tracks = set()
    for item in set_user:
        temp = item + "/tracks"
        driver.get(temp)

        wait(driver, selector_name='sc-link-primary soundTitle__title sc-link-dark sc-text-h4')

        scroll(driver)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        link_tracks = soup.find_all('a', {'class': 'sc-link-primary soundTitle__title sc-link-dark sc-text-h4'})

        for link in link_tracks:
            string_temp = 'https://soundcloud.com'
            string_temp = string_temp + link['href']
            set_link_tracks.add(string_temp)
        for link in set_link_tracks:
            driver.get(link)
            wait(driver, selector_name='sc-ministats-item')
            soup = BeautifulSoup(driver.page_source, "html.parser")
            id_track = soup.find('meta', {'property': 'twitter:app:url:googleplay'})['content'].split(':')[-1]
            name_track = soup.find('h1', {'class': 'soundTitle__title sc-font g-type-shrinkwrap-inline g-type-shrinkwrap-large-primary theme-dark'}).getText().strip()
            time_release_track = soup.find('time', {'class': 'relativeTime'})['title']
            info_track = soup.find_all('span', {'aria-hidden': 'true'})
            plays = info_track[1].getText()
            likes = info_track[2].getText()
            reposts = info_track[3].getText()

            tup = [item[0], id_track, name_track, time_release_track, plays, likes, reposts]
            set_tracks.add(tup)

    return set_tracks


In [ ]:
set_tracks = getTracks(set_link)

In [ ]:
df = pd.DataFrame(set_tracks, columns=['User ID', 
                                          'Tracks ID', 
                                          'Tracks Name', 
                                          'Time_Release_Track', 
                                          'Plays', 
                                          'Likes', 
                                          'Repost'])
df.to_csv('trackHTML.csv')


In [ ]:
#Print All tracks from one persion into file 'playlists.csv'

In [ ]:
# Playlist - parse html func start
def getTracksFromPlaylists(playlist_url, optional):
    driver = webdriver.Chrome()
    driver.get(playlist_url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    user_name = soup.find("a", {"class": "userBadge__usernameLink"}).getText().strip()
    playlist_title = soup.find("h1", {"class": "soundTitle__title"}).getText().strip()
    num_of_track = soup.find("div", {"class": "genericTrackCount__title"}).getText()
    create_date = soup.find("time", {"class": "relativeTime"})["title"]
    datas = soup.find_all("span", {"aria-hidden": "true"})

    likes = datas[1].getText()
    reports = datas[2].getText()

    track_list = []
    track_str = []

    if optional == 0:
        track_items = soup.find_all("div", {"class": "trackItem__content"})

        for track_item in track_items:
            track_name = track_item.getText();
            track_list.append(track_name.strip().replace("\n", " "))
    elif optional == 1:
        track_items = soup.find_all("a", {"class": "trackItem__trackTitle"})
        track_urls = []

        for track_item in track_items:
            track_urls.append('https://soundcloud.com' + track_item['href'])

        for url in track_urls:
            driver = webdriver.Chrome()
            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            meta = soup.find("meta", property="twitter:app:url:iphone")
            track_list.append(meta["content"].split(':')[-1])

    for track_item in track_list:
        track_str.append(track_item)
    track_str = ','.join(track_str)
    if track_str == '':
        track_str = 'None'

    return user_name, playlist_title, num_of_track, create_date, likes, reports, track_str


In [ ]:
def getPlaylistURLs(user_playlist_url):
    url = user_playlist_url
    driver = webdriver.Chrome()
    driver.get(url)
    scroll(driver)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    contents = soup.find_all("a", {"class": "sound__coverArt"})
    urls = []

    for content in contents:
        urls.append('https://soundcloud.com' + content['href'])

    return urls

In [ ]:
def getPlaylists(user_playlist_urls, optional):
    playlist_lst = []

    for user_url in user_playlist_urls:
        playlist_urls = getPlaylistURLs(user_url + '/sets')
        for playlist_url in playlist_urls:
            playlist_lst.append(getTracksFromPlaylists(playlist_url, optional))

    return playlist_lst
# Playlist - parse html func end

In [ ]:
#Print All Playlist into file 'playlists.csv'